In [ ]:
# MIE1621 Fall 2017
# Project 

import numpy as np
import matplotlib.pyplot as plt

def pprint(obj, shapeOnly=True):
    '''
    For debugging, print statements with numpy variable names and shape
    '''
    def namestr(obj):
        namespace = globals()
        return [name for name in namespace if namespace[name] is obj]
    # Assumes obj is a numpy array, matrix
    try:
        print(namestr(obj), obj.shape)
    except:
        try:
            print(namestr(obj), ",", len(obj))
        except:
            print(namestr(obj))
    if not shapeOnly:
        print(obj)


covarianceMatrix = np.array([[0.02778, 0.00387, 0.00021],
                             [0.00387, 0.01112, -0.00020],
                             [0.00021, -0.00020, 0.00115]])

expectedReturn = np.array([0.1073, 0.0737, 0.0627])

stepSize = 1.0

delta = 4.0

maxIteration = 100

# Randomly initialize to anything
initialX = np.array([0.25, 0.25, 0.5])
initialPi = 1.2

def checkShape(x, u, sigma, delta):
    if delta <= 0.0:
        raise ValueError("Risk aversion, delta needs to be > 0")
    if delta <= 3.5 or delta >= 4.5:
        raise ValueError("Risk aversion, delta needs to be between (3.5, 4.5)")
    if sigma.shape[0] != sigma.shape[1]:
        raise ValueError("Covariance matrix must be square")
    if sigma.shape[0] != x.shape[0]:
        raise ValueError("Dimensions dont match between sigma and x")
    if u.shape[0] != x.shape[0]:
        raise ValueError("Dimensions dont match between u and x")
    '''
    Not always met, only met after training is over
    if np.sum(x) != 1.0:
        raise ValueError("X must sum to 1.0")
    '''

def computeF(x, u, sigma, delta):
    '''
    x is the proportion, the parameters we are changing to maximize f
    u is the expected return
    sigma is the covariance matrix
    delta is the parameter that controls risk aversion, delta > 0
    '''
    checkShape(x, u, sigma, delta)
    
    firstTerm = 0.0
    for (ui, xi) in zip(u, x):
        firstTerm += ui * xi
    secondTerm = 0.0
    for i in range(x.shape[0]):
        for j in range(x.shape[0]):
            secondTerm += sigma[i][j] * x[i] * x[j]
    secondTerm *= (float(delta)/float(2.0))
            
    f = firstTerm - secondTerm
    return f

def computeGrad(x, pi, u, sigma, delta):
    checkShape(x, u, sigma, delta)
    grad = np.zeros((x.shape[0] + 1, 1))
    dLdx = np.zeros(x.shape)
    for i in range(x.shape[0]): 
        secondIndex = (i + 1) % x.shape[0]
        thirdIndex = (i + 2) % x.shape[0]
        
        dLdx[i] = u[i] + pi - ((delta)*((float(sigma[i][i]*x[i])/2.0) \
                                        + sigma[i][secondIndex]*x[secondIndex]) \
                               + sigma[i][thirdIndex] * x[thirdIndex])
        grad[i] = dLdx[i]
        
    dLdPi = np.sum(x) - 1.0
    grad[x.shape[0]] = dLdPi
    
    return grad; 

def computeHessian(x, pi, u, sigma, delta):
    checkShape(x, u, sigma, delta)
    dim = x.shape[0] + 1
    hessian = np.zeros((dim, dim))
    for i in range(x.shape[0]): 
        for j in range(x.shape[0]): 
            hessian[i][j] = (-1.0) * delta * sigma[i][j]
            if i == j:
                hessian[i][j] /= 2.0
    for i in range(x.shape[0]): 
        hessian[x.shape[0]][i] = 1.0
        hessian[i][x.shape[0]] = 1.0
    return hessian

x = initialX.copy()
pi = initialPi 
f = computeF(x, expectedReturn, covarianceMatrix, delta)

pprint(covarianceMatrix, False)
pprint(expectedReturn, False)
pprint(x)
# pprint(f)

grad = computeGrad(x, pi, expectedReturn, covarianceMatrix, delta)
pprint(grad)

hessian = computeHessian(x, pi, expectedReturn, covarianceMatrix, delta)
pprint(hessian)


In [ ]:
# Newton Method
x = initialX.copy()
pi = initialPi 
currX = x.copy()
currPi = pi

for iteration in range(maxIteration):
    x = currX.copy()
    pi = currPi
    f = computeF(x, expectedReturn, covarianceMatrix, delta)
    constraint = np.sum(x)
    pprint(iteration, False)
    pprint(f, False)
    pprint(constraint, False)
    pprint(x, False)
    pprint(pi, False)
    
    grad = computeGrad(x, pi, expectedReturn, covarianceMatrix, delta)
    hessian = computeHessian(x, pi, expectedReturn, covarianceMatrix, delta)
    
    direction = np.dot(np.linalg.inv(hessian), grad)
    pprint(direction, False)
    
    # Stopping condition
    if np.linalg.norm(direction) < np.power(0.1, 10):
        break
    xAdd = np.reshape(direction[:-1].copy(), currX.shape)
    
    # You always deduct in Newton Method!
    # As newton method accounts for correct direction regardless of max or min.
    # Newton's method always searches for critical points. 
    currX -= stepSize * xAdd
    currPi -= stepSize * direction[-1]

In [ ]:
# Steepest Descent Method
# Note: NOT really steepest as the question asks to use stepsize of 1.0, so don't calculate stepsize. 
# Just do the normal gradient descent in the Machine Learning literature
x = initialX.copy()
pi = initialPi 
currX = x.copy()
currPi = pi
for iteration in range(maxIteration):
    
    x = currX.copy()
    pi = currPi
    
    f = computeF(x, expectedReturn, covarianceMatrix, delta)
    constraint = np.sum(x)
    pprint(iteration, False)
    pprint(f, False)
    pprint(constraint, False)
    pprint(x, False)
    pprint(pi, False)
    
    grad = computeGrad(x, pi, expectedReturn, covarianceMatrix, delta)
    # No hessian computation needed for gradient descent
    
    direction = grad
    pprint(direction, False)
    
    # Stopping condition
    if np.linalg.norm(direction) < np.power(0.1, 10):
        break
    xAdd = np.reshape(direction[:-1].copy(), currX.shape)
    
    # Gradient Descent diverges with constant step size
    # stepSize = 0.00001 , can only converge with small step size but not OPTIMAL
    # Need to add cause maximizing the function, NOT minimizing
    currX += stepSize * xAdd
    currPi += stepSize * direction[-1]


In [ ]:
# Quasi Newton Methods 
# BFGS
x = initialX.copy()
pi = initialPi 

def BFGSUpdate(H, y, s): 
    Hnext = H + (1.0/(np.dot(np.transpose(y), s))) * np.dot(y, np.transpose(y)) - (1.0/(np.dot(np.dot(np.transpose(s), H), s))) * np.dot(np.dot(np.dot(H, s), np.transpose(s)), H)
    return Hnext

H = np.eye(x.shape[0] + 1)
xnew = np.zeros((x.shape[0]+1,1))
for i in range(x.shape[0]):
    xnew[i] = x[i]
xnew[x.shape[0]] = pi

pprint(xnew, False)
for iteration in range(maxIteration):
    xold = xnew
    gradFk = computeGrad(xold[:-1], xold[-1], expectedReturn, covarianceMatrix, delta)
    direction = np.dot(np.linalg.inv(H), gradFk)
    # Stopping condition
    if np.linalg.norm(direction) < np.power(0.1, 10):
        break
    #----------------------------------------------------------
    xnew = xold - stepSize * direction
    gradFk1 = computeGrad(xnew[:-1], xnew[-1], expectedReturn, covarianceMatrix, delta)
    s = xnew - xold
    y = gradFk1 - gradFk
    # BFGS Update
    Hnext = BFGSUpdate(H,y,s)
    H = Hnext
    
    f = computeF(xnew[:-1], expectedReturn, covarianceMatrix, delta)
    pprint(iteration, False)
    pprint(f, False)
    pprint(direction, False)
    pprint(xnew, False)
    '''
    pprint(xold)
    pprint(H)
    pprint(gradFk)
    pprint(gradFk1)
    pprint(y)
    pprint(s)
    '''

In [ ]:
# Backtracking 

# From wolfe's theorem to calculate backtracking
# 0 < gamma < beta < 1
gamma = np.power(10.0, -4) # Common for gamma to be within [0, 0.5] # Close to 0
#gamma = 0.5
beta = 0.9 # Close to 1

def checkGammaBeta(gamma, beta):
    if gamma <= 0.0:
        raise ValueError("Gamma must be > 0.0")
    if beta >= 1.0:
        raise ValueError("Beta must be < 1.0")
    if gamma >= beta:
        raise ValueError("Gamma must be < Beta")
    return

def backtrack(stepSize, gamma, beta, direction, gradOld, gradNew, fOld, fNew):
    '''
    Returns true if stepsize works
    '''
    checkGammaBeta(gamma, beta)
    firstConditionLhs = np.dot(np.transpose(direction), gradNew)
    firstConditionRhs = beta * np.dot(np.transpose(direction), gradOld)
    # Condition iii) in notes
    if firstConditionLhs > firstConditionRhs:
        secondConditionLhs = fNew
        secondConditionRhs = fOld + gamma * stepSize * np.dot(np.transpose(direction), gradOld)
        secondConditionRhs = secondConditionRhs[0][0]
        
        #print("SECOND CONDITIONLHS", secondConditionLhs)
        #print("SECOND CONDITIONRHS", secondConditionRhs)
        # Condition iv) in notes
        if secondConditionLhs > secondConditionRhs:
            # If satisfy condition, return
            return True
    return False

In [ ]:
# Newton Method with Backtracking
currX = x.copy()
currPi = pi
for iteration in range(maxIteration):
    x = currX.copy()
    pi = currPi
    f = computeF(x, expectedReturn, covarianceMatrix, delta)
    constraint = np.sum(x)
    pprint(iteration, False)
    pprint(f, False)
    pprint(constraint, False)
    pprint(x, False)
    pprint(pi, False)
    
    grad = computeGrad(x, pi, expectedReturn, covarianceMatrix, delta)
    hessian = computeHessian(x, pi, expectedReturn, covarianceMatrix, delta)
    
    direction = np.dot(np.linalg.inv(hessian), grad)
    pprint(direction, False)
    
    # 
    direction *= -1.0
    tempDirection = direction
    value = np.dot(np.transpose(tempDirection), grad)
    value = value[0][0]
    # Use less than due to maximization
    if value < 0:
        print("Value: ", value)
        direction *= -1.0
        tempDirection = direction
        value = np.dot(np.transpose(tempDirection), grad)
        value = value[0][0]
        print("Value: ", value)
        if value < 0:
            raise ValueError("value does not satisfy less than 0 condition")

    # Stopping condition
    if np.linalg.norm(direction) < np.power(0.1, 10):
        print("Met stopping point, breaking!")
        break
    xAdd = np.reshape(direction[:-1].copy(), currX.shape)
    stepSize = 1.0
    while stepSize > 0.0:
        tempCurrX = currX + stepSize * xAdd
        tempCurrPi = currPi + stepSize * direction[-1]
        
        newGrad = computeGrad(tempCurrX, tempCurrPi, expectedReturn, covarianceMatrix, delta)
        newF = computeF(tempCurrX, expectedReturn, covarianceMatrix, delta)
        if backtrack(stepSize, gamma, beta, tempDirection, grad, newGrad, f, newF):
            break
        # Maximum number of steps
        stepSize -= 0.001
    pprint(stepSize, False)
    #'''
    if stepSize <= 0.0:
        raise ValueError("Stepsize doesnt work")
    #'''
    currX += stepSize * xAdd
    currPi += stepSize * direction[-1]
    
    
    # You always deduct in Newton Method!
    # As newton method accounts for correct direction regardless of max or min.
    # Newton's method always searches for critical points. 

In [ ]:
# Steepest Descent Method with Backtracking
# Steepest Descent Method
# Note: NOT really steepest as the question asks to use stepsize of 1.0, so don't calculate stepsize. 
# Just do the normal gradient descent in the Machine Learning literature
x = initialX.copy()
pi = initialPi 
currX = x.copy()
currPi = pi
for iteration in range(maxIteration):
    
    x = currX.copy()
    pi = currPi
    
    f = computeF(x, expectedReturn, covarianceMatrix, delta)
    constraint = np.sum(x)
    pprint(iteration, False)
    pprint(f, False)
    pprint(constraint, False)
    pprint(x, False)
    pprint(pi, False)
    
    grad = computeGrad(x, pi, expectedReturn, covarianceMatrix, delta)
    # No hessian computation needed for gradient descent
    
    direction = grad.copy()
    direction *= -1.0
    pprint(direction, False)
    tempDirection = direction 
    value = np.dot(np.transpose(tempDirection), grad)
    value = value[0][0]
    if value <= 0:
        print("Value: ", value)
        direction *= -1.0
        tempDirection = direction
        pprint(direction, False)
        value = np.dot(np.transpose(tempDirection), grad)
        value = value[0][0]
        print("Value: ", value)
        if value <= 0:
            raise ValueError("value does not satisfy less than 0 condition")
        
    
    # Stopping condition
    if np.linalg.norm(direction) < np.power(0.1, 10):
        break
    xAdd = np.reshape(direction[:-1].copy(), currX.shape)
    
    stepSize = 1.0
    while stepSize > 0.0:
        tempCurrX = currX + stepSize * xAdd
        tempCurrPi = currPi + stepSize * direction[-1]
        
        newGrad = computeGrad(tempCurrX, tempCurrPi, expectedReturn, covarianceMatrix, delta)
        newF = computeF(tempCurrX, expectedReturn, covarianceMatrix, delta)
        if backtrack(stepSize, gamma, beta, tempDirection, grad, newGrad, f, newF):
            break
        # Maximum number of steps
        stepSize -= 0.0001
    pprint(stepSize, False)
    #'''
    if stepSize <= 0.0:
        raise ValueError("Stepsize doesnt work")
    #'''
    
    # Gradient Descent diverges with constant step size
    # stepSize = 0.00001 , can only converge with small step size but not OPTIMAL
    # Need to add cause maximizing the function, NOT minimizing
    currX += stepSize * xAdd
    currPi += stepSize * direction[-1]

In [ ]:
# Quasi Newton Methods with Backtracking
# BFGS


In [ ]:
# Scaling
# Reading in file 

projFolder = "./projectFiles/"
extension = ".csv"

SamsungFile = projFolder + "Samsung" + extension
FacebookFile = projFolder + "Facebook" + extension
AMDFile = projFolder + "AMD" + extension
SalesforceFile = projFolder + "Salesforce" + extension
adjustedCloseIndex = 5

maxIteration = 1000
stepSize = 0.01

def getClosingPrice(fileName):
    firstLine = True
    output = open(fileName, "r")
    closingPrices = []
    for line in output:
        if firstLine:
            firstLine = False
            continue
        values = line.split(",")
        closingPrice = float(values[adjustedCloseIndex])
        closingPrices.append(closingPrice)
    return np.array(closingPrices)

def getDailyReturns(closingPrice):
    # Deduct 1 from shape as always need 2 days to get daily return
    dailyReturns = np.zeros(closingPrice.shape[0] - 1)
    for i in range(dailyReturns.size):
        dailyReturns[i] = closingPrice[i+1] - closingPrice[i]
    return dailyReturns

def calculateExpectedReturn(arrOfClosingPrices):
    expectedReturnPerStock = np.mean(arrOfClosingPrices, axis = 1)
    return expectedReturnPerStock

def calculateCovarianceMatrix(arrOfClosingPrices):
    numberOfClosingPrices = arrOfClosingPrices.shape[1]
    numberOfStocks = arrOfClosingPrices.shape[0] 
    expectedReturnPerStock = np.mean(arrOfClosingPrices, axis = 1)
    covarianceMatrix = np.zeros((expectedReturnPerStock.shape[0], expectedReturnPerStock.shape[0]))
    for i in range(numberOfStocks):
        for j in range(numberOfStocks):
            covarianceMatrix[i][j] =  0
            sum = 0.0
            for k in range(numberOfClosingPrices):
                firstStock = arrOfClosingPrices[i][k] - expectedReturnPerStock[i]
                secondStock = arrOfClosingPrices[j][k] - expectedReturnPerStock[j] 
                sum += firstStock * secondStock
            covarianceMatrix[i][j] = float(sum)/float(numberOfClosingPrices - 1)
    return covarianceMatrix

samsungClosingPrices = getClosingPrice(SamsungFile)
facebookClosingPrices = getClosingPrice(FacebookFile)
amdClosingPrices = getClosingPrice(AMDFile)
salesforceClosingPrices = getClosingPrice(SalesforceFile)

# Truncate to the minimum size
minSize = samsungClosingPrices.size
minSize = min(minSize, samsungClosingPrices.size)
minSize = min(minSize, facebookClosingPrices.size)
minSize = min(minSize, amdClosingPrices.size)
minSize = min(minSize, salesforceClosingPrices.size)
samsungClosingPrices = samsungClosingPrices[:minSize]
facebookClosingPrices = facebookClosingPrices[:minSize]
amdClosingPrices = amdClosingPrices[:minSize]
salesforceClosingPrices = salesforceClosingPrices[:minSize]
pprint(samsungClosingPrices)
pprint(facebookClosingPrices)
pprint(amdClosingPrices)
pprint(salesforceClosingPrices)

# Get daily returns

samsungDailyReturns = getDailyReturns(samsungClosingPrices)
facebookDailyReturns = getDailyReturns(facebookClosingPrices)
amdDailyReturns = getDailyReturns(amdClosingPrices)
salesforceDailyReturns = getDailyReturns(salesforceClosingPrices)
pprint(samsungDailyReturns)
pprint(facebookDailyReturns)
pprint(amdDailyReturns)
pprint(salesforceDailyReturns)

allReturns = []
allReturns.append(samsungDailyReturns)
allReturns.append(facebookDailyReturns)
allReturns.append(amdDailyReturns)
allReturns.append(salesforceDailyReturns)
allReturns = np.array(allReturns)
pprint(allReturns)

expectedReturn = calculateExpectedReturn(allReturns)
pprint(expectedReturn, False)
covarianceMatrix = calculateCovarianceMatrix(allReturns)
# Calculate each mean
pprint(covarianceMatrix, False)

initialX = np.array([0.25, 0.25, 0.25, 0.25])
initialPi = 1.2

In [ ]:
# Newton Method
x = initialX.copy()
pi = initialPi 
currX = x.copy()
currPi = pi

for iteration in range(maxIteration):
    x = currX.copy()
    pi = currPi
    
    f = computeF(x, expectedReturn, covarianceMatrix, delta)
    constraint = np.sum(x)
    pprint(iteration, False)
    pprint(f, False)
    pprint(constraint, False)
    pprint(x, False)
    pprint(pi, False)
    
    grad = computeGrad(x, pi, expectedReturn, covarianceMatrix, delta)
    hessian = computeHessian(x, pi, expectedReturn, covarianceMatrix, delta)
    
    direction = np.dot(np.linalg.inv(hessian), grad)
    pprint(direction, False)
    
    # Stopping condition
    if np.linalg.norm(direction) < np.power(0.1, 10):
        break
    xAdd = np.reshape(direction[:-1].copy(), currX.shape)
    
    # You always deduct in Newton Method!
    # As newton method accounts for correct direction regardless of max or min.
    # Newton's method always searches for critical points. 
    currX -= stepSize * xAdd
    currPi -= stepSize * direction[-1]

In [ ]:
# Steepest Descent Method
# Note: NOT really steepest as the question asks to use stepsize of 1.0, so don't calculate stepsize. 
# Just do the normal gradient descent in the Machine Learning literature
x = initialX.copy()
pi = initialPi 
currX = x.copy()
currPi = pi
for iteration in range(maxIteration):
    
    x = currX.copy()
    pi = currPi
    
    f = computeF(x, expectedReturn, covarianceMatrix, delta)
    constraint = np.sum(x)
    pprint(iteration, False)
    pprint(f, False)
    pprint(constraint, False)
    pprint(x, False)
    pprint(pi, False)
    
    grad = computeGrad(x, pi, expectedReturn, covarianceMatrix, delta)
    # No hessian computation needed for gradient descent
    
    direction = grad
    pprint(direction, False)
    
    # Stopping condition
    if np.linalg.norm(direction) < np.power(0.1, 10):
        break
    xAdd = np.reshape(direction[:-1].copy(), currX.shape)
    
    # Gradient Descent diverges with constant step size
    # stepSize = 0.00001 , can only converge with small step size but not OPTIMAL
    # Need to add cause maximizing the function, NOT minimizing
    currX += stepSize * xAdd
    currPi += stepSize * direction[-1]
